<a href="https://colab.research.google.com/github/raminbazr/Python/blob/main/Hyperparameters_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Finding hperparameters of LSTM model with genetic algorithm**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install pandas_datareader --upgrade
! pip install bitstring
! pip install bitarray 
! pip install deap

In [ ]:
from pandas_datareader import data
from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM, Input, Dense
from keras.layers import Dropout
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split
import math
from keras.layers import LSTM, Input, Dense
from keras.models import Model

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray
from bitarray import bitarray
from bitarray.util import ba2int

np.random.seed(1120)

In [ ]:
def df_to_X_y(train, window_size):
  #df_as_np = data.to_numpy()
  X = []
  y = []
  for i in range(len(train)-window_size):
    row = [r for r in train[i:i+window_size]]
    X.append(row)
    label = [train[i+window_size][4],train[i+window_size][1]]
    y.append(label)
  return np.array(X), np.array(y)

X_train, y_train = df_to_X_y(train,3)
X_test, y_test = df_to_X_y(test,3)

In [ ]:
def train_evaluate(ga_individual_solution):  
    print(ga_individual_solution)

    # Decode the Genetic Algorithm solution to get the window size and number of units and Dropout rate
    window_size_bits = BitArray(ga_individual_solution[0:5])
    num_units_bits = BitArray(ga_individual_solution[5:12])   #7 to neuron number
    Dropout_rate_bits = BitArray(ga_individual_solution[12:]) 
    window_size = window_size_bits.uint
    num_of_units = num_units_bits.uint
    Dropout_rate = ba2int(bitarray(Dropout_rate_bits))/100
    # print(num_units_bits)
    # print(Dropout_rate)
    print('\nWindow Size: ', window_size, ', Num of Units: ', num_of_units, ', DropoutRate:',Dropout_rate)
    
    # Return fitness score of 100 if window_size or num_unit is zero
    if window_size == 0 or num_of_units == 0 :
        return 100,
    
    # Segment the train_data based on new window_size; 
    x_train,y_train = df_to_X_y(train,window_size)
    x_val,y_val = df_to_X_y(test,window_size)

    # X_data,Y_data = prepare_dataset(train_data, window_size)
    # x_train, x_val, y_train, y_val = split(X_data, Y_data, test_size = 0.10, random_state = 1120)
    
    # Design an LSTM model to train on training data and predict on validation data
    regressor = Sequential()
    
    regressor.add(LSTM(units = num_of_units, return_sequences = True, input_shape = (window_size, 15)))
    regressor.add(Dropout(Dropout_rate))
    
    regressor.add(LSTM(units = num_of_units, return_sequences = True))
    regressor.add(Dropout(Dropout_rate))
    
    #regressor.add(LSTM(units = num_of_units, return_sequences = True))
    #regressor.add(Dropout(Dropout_rate))
    
    regressor.add(LSTM(units = num_of_units))
    regressor.add(Dropout(Dropout_rate))
    
    regressor.add(Dense(units = 2))
    
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    regressor.fit(x_train, y_train, epochs=100, batch_size=64,shuffle=True)
    y_pred = regressor.predict(x_val)
    
    # Calculate the RMSE score as fitness score for GA
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print('Validation RMSE: ', rmse,'\n') 
    return rmse,

In [ ]:
#taguchi =[2 ,0.1, 0.1,	0.1]
#taguchi =[2 ,0.4,	0.5,	0.3]
#taguchi =[2	,0.8, 0.8,	0.6]
#taguchi =[4	,0.1, 0.5,	0.6]
#taguchi =[4	,0.4, 0.8,	0.1]
#taguchi =[4	,0.8, 0.1,	0.3]
#taguchi =[8	,0.1, 0.8,	0.3]
#taguchi =[8	,0.4, 0.1,	0.6]
#taguchi =[8	,0.8, 0.5,	0.1]
taguchi=[2, 0.1, 0.5, 0.1]

population_size = taguchi[0]  #4 choromosome
num_generations = 4
gene_length = 17     #each choromosome consist of 17 bit


# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = taguchi[3])
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = taguchi[1], mutpb = taguchi[2], ngen = num_generations, verbose = False)

In [ ]:
optimal_individuals_data = tools.selBest(population,k = 3) #select top 1 solution
optimal_window_size = None
optimal_num_units = None
optimal_Dropout_rate = None

for bi in optimal_individuals_data:
    window_size_bits = BitArray(bi[0:5])
    num_units_bits = BitArray(bi[5:12])
    Dropout_rate_bits = BitArray(bi[12:]) 
    optimal_window_size = window_size_bits.uint
    optimal_num_units = num_units_bits.uint
    optimal_Dropout_rate = ba2int(bitarray(Dropout_rate_bits))/100
    print('\n Best Window Size: ', optimal_window_size, ', Best Num of Units: ', optimal_num_units, ', Best Dropout Rate: ', optimal_Dropout_rate)



In [ ]:
# train the model with the optimal number of lstm units and optimal Dropout rate and optimal window size for prediction
num_of_units = optimal_num_units
Dropout_rate = optimal_Dropout_rate
window_size = optimal_window_size

regressor = Sequential()
    
regressor.add(LSTM(units = num_of_units, return_sequences = True, input_shape = (3,1))) #windows_size gets 3 ramin
regressor.add(Dropout(Dropout_rate))

regressor.add(LSTM(units = num_of_units, return_sequences = True))
regressor.add(Dropout(Dropout_rate))

#regressor.add(LSTM(units = num_of_units, return_sequences = True))
#regressor.add(Dropout(Dropout_rate))

regressor.add(LSTM(units = num_of_units))
regressor.add(Dropout(Dropout_rate))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

history = regressor.fit(X_train, y_train, epochs=30, batch_size=64,shuffle=True)
x_val,y_val = prepare_dataset(test,3)
y_pred = regressor.predict(x_val)

# Calculate the RMSE score as fitness score for GA
rmse = np.sqrt(mean_squared_error(y_val, y_pred[:,0]))
print('Validation RMSE: ', rmse,'\n') 

In [ ]:
# plt.plot(np.arange(0, epochs), history.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, 30), history.history['loss'], label='train_loss')
plt.title('Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch #')
plt.legend()
plt.show()

In [ ]:
#hence train the model with the optimal number of lstm units and optimal Dropout rate and optimal window size for prediction
x_test,y_test = prepare_dataset(test,3)

y_pred = regressor.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred[:,0]))
print('Test RMSE: ', rmse)

In [ ]:
plt.plot(test, color = 'black', label = 'ROPc')
plt.plot(y_pred, color = 'green', label = 'Predicted ROPc')
plt.title('ROPc Prediction')
plt.xlabel('Time')
plt.ylabel('ROP')
plt.legend()
plt.show()